In [1]:
import numpy as np
import pandas as pd
import os

In [1]:
# pip install IMDbPY
# from imdb import IMDb

# # create an instance of the IMDb class
# ia = IMDb()

# # get a movie and print its director(s)
# the_matrix = ia.get_movie('0133093')
# for director in the_matrix['directors']:
#     print(director['name'])

# # show all information that are currently available for a movie
# print(sorted(the_matrix.keys()))

# # show all information sets that can be fetched for a movie
# print(ia.get_movie_infoset())

Lana Wachowski
Lilly Wachowski
['akas', 'animation department', 'art department', 'art directors', 'aspect ratio', 'assistant directors', 'box office', 'camera department', 'canonical title', 'cast', 'casting department', 'casting directors', 'certificates', 'cinematographers', 'color info', 'composers', 'costume departmen', 'costume designers', 'countries', 'country codes', 'cover url', 'director', 'directors', 'distributors', 'editorial department', 'editors', 'full-size cover url', 'genres', 'kind', 'language codes', 'languages', 'location management', 'long imdb canonical title', 'long imdb title', 'make up department', 'miscellaneous', 'music department', 'original air date', 'other companies', 'plot', 'plot outline', 'producers', 'production companies', 'production designers', 'production managers', 'rating', 'runtimes', 'script department', 'set decorators', 'smart canonical title', 'smart long imdb canonical title', 'sound department', 'sound mix', 'special effects', 'special e

In [2]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn

headers = {"Accept-Language": "en-US, en;q=0.5"}

In [319]:
##################
# IMDB Data Scrape
##################

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# declaring the lists to store data in
name = []
year = []
rating = []
metascore = []
vote = []
imdb_id = []
poster = []
genre = []
certificate = []
runtime = []
director_actor = []
story = []

# 12,653 Korean Movie/TV on IMDB - 254 Pages
# pages = [i for i in range(0,90)]
# pages = [i for i in range(90,180)]
pages = [i for i in range(180,255)]

# For every page in the interval 1-4
for page in pages:

    # Make a get request
    response = get('http://www.imdb.com/search/title?country_of_origin=kr&start=' + str(page*50+1) +
    '&ref_=adv_nxt', headers = headers)

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > 99:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the 50 movie containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

    # For every movie of these 50
    for container in mv_containers:
        # If the movie has a Metascore, then:
        # if container.find('div', class_ = 'ratings-metascore') is not None:

        # name
        if container.h3.a:
            name.append(container.h3.a.text)
        else: name.append(np.nan)

        # year
        if container.h3.find('span', class_='lister-item-year'):
            year.append(container.h3.find('span', class_='lister-item-year').text)
        else: year.append(np.nan)

        # IMDB rating
        if container.strong:
            rating.append(float(container.strong.text))
        else: rating.append(np.nan)

        # Metascore
        if container.find('span', class_='metascore'):
            metascore.append(int(container.find('span', class_='metascore').text))
        else: metascore.append(np.nan)

        # number of votes
        if container.find('span', attrs={'name':'nv'}):
            vote.append(int(container.find('span', attrs={'name':'nv'})['data-value'].replace(',','')))
        else: vote.append(np.nan)

        # movie id
        if container.find('img').get('data-tconst'):
            imdb_id.append(container.find('img').get('data-tconst'))
        else: imdb_id.append(np.nan)

        # poster link
        if container.find('img').get('loadlate'):
            poster.append(container.find('img').get('loadlate'))
        else: poster.append(np.nan)

        #Scrape the genre
        if container.find('span',attrs={'class':'genre'}):
            genre.append(container.find('span',attrs={'class':'genre'}).text)
        else: genre.append(np.nan)

        # certificate
        if container.find('span',attrs={'class':'certificate'}):
            certificate.append(container.find('span',attrs={'class':'certificate'}).text)
        else: certificate.append(np.nan)

        # runtime
        if container.find('span',attrs={'class':'runtime'}):
            runtime.append(container.find('span',attrs={'class':'runtime'}).text)
        else: runtime.append(np.nan)

        # director_actor
        if container.find_all('p'):
            director_actor.append(container.find_all('p')[2].text)
        else: director_actor.append(np.nan)

        # Scrape the story
        if container.find_all('p'):
            story.append(container.find_all('p')[1].text)
        else: story.append(np.nan)

Request:75; Frequency: 0.07579557774754757 requests/s


In [320]:
# Assemble movie rating table
movie_ratings = pd.DataFrame({'name': name,
                              'year': year,
                              'rating': rating,
                              'metascore': metascore,
                              'vote': vote,
                              'imdb_id': imdb_id,
                              'poster': poster,
                              'genre': genre,
                              'certificate': certificate,
                              'runtime': runtime,
                              'director_actor': director_actor,
                              'story': story
                             })
print(movie_ratings.info())
movie_ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Data columns (total 12 columns):
name              3750 non-null object
year              3750 non-null object
rating            2682 non-null float64
metascore         1155 non-null float64
vote              2682 non-null float64
imdb_id           3750 non-null object
poster            3750 non-null object
genre             3649 non-null object
certificate       1762 non-null object
runtime           3380 non-null object
director_actor    3750 non-null object
story             3750 non-null object
dtypes: float64(3), object(9)
memory usage: 351.6+ KB
None


,name,year,rating,metascore,vote,imdb_id,poster,genre,certificate,runtime,director_actor,story
0,Pickled Radishes,(1964),NaN,NaN,NaN,tt0353299,https://m.media-amazon.com/images/G/01/imdb/im...,"\nDrama, Romance",NaN,100 min,\n Director:\nIl-Ho Jang\n| \n Stars:\nS...,\n Two childhood friends look for suitable ...
1,A Man's World,(I) (2012),NaN,NaN,NaN,tt2398113,https://m.media-amazon.com/images/M/MV5BMjIzNT...,NaN,NaN,NaN,\n Director:\nThomas Maitland\n| \n Star...,\n A father and his son navigate the mores ...
2,LYN: The Lightbringer,(2019 Video Game),NaN,NaN,NaN,tt10980800,https://m.media-amazon.com/images/G/01/imdb/im...,"\nAnimation, Action, Adventure",NaN,NaN,"\n \n Stars:\nKira Buckland, \nT...",\n Light and Darkness came into conflict on...
3,Traitor Shanghia Park,(1965),NaN,NaN,NaN,tt0326748,https://m.media-amazon.com/images/G/01/imdb/im...,NaN,NaN,NaN,\n Director:\nWon-jik Lim\n| \n Stars:\n...,\n A Korean man works for the Japanese colo...
4,Yegichi mothan sarang,(1965),NaN,NaN,NaN,tt0410923,https://m.media-amazon.com/images/G/01/imdb/im...,"\nDrama, Romance",NaN,NaN,\n Director:\nJong-ho Park\n| \n Stars:\...,\n Kyeong-rim falls in love with an older m...


In [401]:
# movie_ratings.to_csv('movie_ratings_1_4500.csv', encoding='utf-8-sig')
# movie_ratings_1_4500 = movie_ratings

# movie_ratings.to_csv('movie_ratings_4501_9000.csv', encoding='utf-8-sig')
# movie_ratings_4501_9000 = movie_ratings

# movie_ratings.to_csv('movie_ratings_9001_12750.csv', encoding='utf-8-sig')
# movie_ratings_9001_12750 = movie_ratings

# imdb_kr = movie_ratings_1_4500
# imdb_kr = imdb_kr.append(movie_ratings_4501_9000, ignore_index=True)
# imdb_kr = imdb_kr.append(movie_ratings_9001_12750, ignore_index=True)

# # remove duplicate by imdb_id
# imdb_kr.sort_values("imdb_id",inplace=True)
# imdb_kr.drop_duplicates(subset="imdb_id",inplace=True)
# len(imdb_kr)

12750

In [414]:
# # create numeric imdb id
# imdb_kr['imdbId'] = imdb_kr['imdb_id'].apply(lambda s:int(s[2:]))

# # raw data path
# data_raw_p = os.path.abspath('../data/raw')

# # save a csv copy and a pickle copy to raw data path
# imdb_kr.to_csv(data_raw_p+'/imdb_kr.csv', index=False, header=True, encoding='utf-8-sig')
# imdb_kr.to_pickle(data_raw_p+'/imdb_kr.pickle')

In [48]:
#######################
# connect to SQL engine
#######################
from sqlalchemy import create_engine, event
from sqlalchemy_utils import database_exists, create_database
import psycopg2

engine = create_engine('postgresql://happyt3@localhost/mediaquant')

# declare a new PostgreSQL connection object
conn = psycopg2.connect(
    dbname = "mediaquant",
    user = "happyt3",
    host = "localhost")

# @event.listens_for(engine, 'before_cursor_execute')
# def receive_before_cursor_execute(connection, cursor, statement, params, context, executemany):
#     # print("FUNC call")
#     if executemany:
#         cursor.fast_executemany = True
#         cursor.commit()
        
def execute_query(connection, query):
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed successfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")

In [49]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print("Table: %s" % table_name)
#         for column in inspector.get_columns(table_name, schema=schema):
#             print("Column: %s" % column)

schema: information_schema
Table: sql_packages
Table: sql_features
Table: sql_implementation_info
Table: sql_parts
Table: sql_languages
Table: sql_sizing
Table: sql_sizing_profiles
schema: public
Table: imdb_kr
Table: ml_links
Table: youtube_movie_trailer_comments
Table: ml_ratings
Table: ml_ratings_temp


In [9]:
# engine.execute("DROP TABLE IF EXISTS ml_ratings_temp")
# engine.execute("DROP TABLE IF EXISTS ml_ratings")

In [15]:
# # create imdb_kr_table
# create_imdb_kr_table = """
# CREATE TABLE IF NOT EXISTS imdb_kr (
#   name TEXT NOT NULL,
#   year TEXT,
#   rating FLOAT,
#   metascore FLOAT,
#   vote FLOAT,
#   imdb_id TEXT,
#   poster TEXT,
#   genre TEXT,
#   certificate TEXT,
#   runtime TEXT,
#   director_actor TEXT,
#   story TEXT,
#   imdbId INTEGER
# )
# """
# execute_query(conn, create_imdb_kr_table)

# # create a temp table
# imdb_kr.to_sql('imdb_kr_temp', con=engine, if_exists='replace', index=False)

# # insert rows from the temp table
# engine.execute("""
# INSERT INTO imdb_kr (name,year,rating,metascore,vote,imdb_id,poster,genre,certificate,runtime,director_actor,story,imdbId)
# SELECT DISTINCT *
# FROM imdb_kr_temp;
# """)

# # drop table
# engine.execute("DROP TABLE IF EXISTS imdb_kr_temp")

# # rename table
# engine.execute("ALTER TABLE imdb RENAME TO imdb_kr_backup")

In [12]:
# count movie id
engine.execute("SELECT count(*) as count FROM imdb_kr").fetchall()

[(10000,)]

In [11]:
################
# MovieLens Data
################

# MovieLens 25M Dataset
# MovieLens 25M movie ratings. Stable benchmark dataset. 25 million ratings and one million tag applications applied to 62,000 movies by 162,000 users. Includes tag genome data with 15 million relevance scores across 1,129 tags. Released 12/2019

# README.txt
# ml-25m.zip (size: 250 MB, checksum)
# Permalink: https://grouplens.org/datasets/movielens/25m/
movielens25m_p = os.path.abspath('../data/raw/ml-25m') # MovieLens 25M Dataset

In [450]:
# # create MovieLens IMDB links table
# create_ml_links_table = """
# CREATE TABLE IF NOT EXISTS ml_links (
#   movieId INTEGER,
#   imdbId INTEGER,
#   tmdbId FLOAT
# )
# """
# execute_query(conn, create_ml_links_table)

# ml_links = pd.read_csv(movielens25m_p+'/links.csv')
# # create a temp sql table
# ml_links.to_sql('ml_links_temp', con=engine, if_exists='replace', index=False)
# # copy rows from ml_links_temp to ml_links
# engine.execute("""
# INSERT INTO ml_links (movieId,imdbId,tmdbId)
# SELECT DISTINCT *
# FROM ml_links_temp;
# """)

# # count movie id
# engine.execute("SELECT count(*) as count FROM ml_links").fetchall()
# engine.execute("DROP TABLE IF EXISTS ml_links_temp")

Query executed successfully


In [16]:
# create MovieLens ratings table
create_ml_ratings_table = """
CREATE TABLE IF NOT EXISTS ml_ratings (
  userId INTEGER,
  movieId INTEGER,
  rating FLOAT,
  timestamp INTEGER
)
"""
execute_query(conn, create_ml_ratings_table)

ml_ratings = pd.read_csv(movielens25m_p+'/ratings.csv')

Query executed successfully


In [45]:
# keep 25M ratings
ml_ratings.drop(ml_ratings.index[25000000:], inplace=True)
chunks = np.split(ml_ratings, ml_ratings.shape[0] / 10**6)

25000000

In [ ]:
# create a temp sql table
# ml_ratings.to_sql('ml_ratings_temp', con=engine, if_exists='replace', index=False) # too slow
# ml_ratings.to_sql('ml_ratings_temp', con=engine, if_exists='replace', index=False,
#                   chunksize=500000, method='multi')

# def write_df_to_sql(df, **kwargs):
#     chunks = np.split(df, df.shape[0] / 10**6)
#     for chunk in chunks:
#         chunk.to_sql(**kwargs)
#     return True

# write_df_to_sql(ml_ratings, 'ml_ratings_temp', con=engine, if_exists='replace', index=False)

for chunk in chunks:
    chunk.to_sql('ml_ratings_temp', con=engine, if_exists='append', index=False)

In [51]:
engine.execute("SELECT count(*) as count FROM ml_ratings").fetchall()

[(0,)]

In [ ]:
# copy rows from ml_ratings_temp to ml_ratings
engine.execute("""
INSERT INTO ml_ratings (userId,movieId,rating,timestamp)
SELECT DISTINCT *
FROM ml_ratings_temp;
""")

# count movie id
engine.execute("SELECT count(*) as count FROM ml_ratings").fetchall()
engine.execute("DROP TABLE IF EXISTS ml_ratings_temp")